In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [3]:
%matplotlib inline

In [4]:
plt.rcParams['figure.figsize'] = (12,8)

In [5]:
# read in all our data
ses_csv = pd.read_csv('./Demographics:SocioeconomicStatus/Demographic_Statistics_By_Zip_Code.csv')
high_school_directory_csv = pd.read_csv('./Graduation:Regents:AP/DOE_High_School_Directory_2014-2015.csv')
grad_outcomes_csv = pd.read_csv('./Graduation:Regents:AP/Graduation_Outcomes_-_Classes_Of_2005-2010_By_Borough.csv')
school_budget_csv = pd.read_csv('./Graduation:Regents:AP/School_Budget_Overview.csv')
fpm_csv = pd.read_csv('./AirQuality/fpm.csv')
attendance_csv = pd.read_csv('./Attendance/Attendance_4PM_Report.csv')

In [6]:
# convert things to datetime
attendance_csv['DATE'] = pd.to_datetime(attendance_csv['DATE'])

In [7]:
# join on district/borough/number identifier for each school
hs_attendance = attendance_csv.merge(high_school_directory_csv, left_on=['SCHOOL_DBN'], right_on=['dbn'], how='inner')

In [8]:
reduced_cols = hs_attendance[['DATE', 'zip', 'dbn', '%_OF_ATTD_TAKEN', 'school_name', 'total_students', 'boro']]

In [9]:
# renaming column to make more sense
ses_csv['zip'] = ses_csv['JURISDICTION NAME']

In [10]:
# merging socioeconomic status with attendance / high school
ses_attd_hs = reduced_cols.merge(ses_csv, on='zip', how='inner').drop('JURISDICTION NAME', axis=1)

In [11]:
# convert air quality to dates
def dateify(x):
    # we have things by season, as "Summer 2014"
    season, years = x.split(' ')
    
    # peg them to the start of the season
    if season == 'Summer':
        month = 6
    elif season == 'Winter':
        month = 12
    y = years[:4]
    # return a reasonable date string
    return '{}/{}'.format(month, y)

# convert them to datetime objects for timeseries analysis
fpm_csv['date'] = pd.to_datetime(fpm_csv['Year'].map(dateify))

In [12]:
# some regex to match identifiers between New York State and NYC
import re
def to_location(x):
    return re.search('\d+([A-Z]\d+)', x).groups(1)[0]
    
ses_attd_hs['bloc'] = ses_attd_hs['dbn'].map(to_location)

budget_ses_attd_hs = school_budget_csv.merge(ses_attd_hs, left_on=['Location'], right_on=['bloc'], how='inner')

In [13]:
ses_attd_hs[ses_attd_hs['boro'] == 'Bronx'].describe()

zip  %_OF_ATTD_TAKEN  total_students  COUNT PARTICIPANTS  \
count  14460.000000     14460.000000    14460.000000        14460.000000   
mean   10461.587690        81.841480      490.590456           31.740249   
std        6.805702        12.667752      413.501922           31.364111   
min    10451.000000         0.000000       71.000000            0.000000   
25%    10456.000000        77.900000      335.000000            5.000000   
50%    10460.000000        83.700000      435.000000           27.000000   
75%    10468.000000        89.500000      523.000000           52.000000   
max    10475.000000       100.000000     3037.000000          106.000000   

       COUNT FEMALE  PERCENT FEMALE    COUNT MALE  PERCENT MALE  \
count  14460.000000    14460.000000  14460.000000  14460.000000   
mean      18.072199        0.447339     13.668050      0.314971   
std       18.113483        0.259352     13.570401      0.189206   
min        0.000000        0.000000      0.000000      0.000000   
25%        3.000000        0.440000      2.000000      0.260000   
50%       17.000000        0.560000      8.000000      0.400000   
75%       26.000000        0.600000     26.000000      0.440000   
max       65.000000        0.740000     41.000000      0.560000   

       COUNT GENDER UNKNOWN  PERCENT GENDER UNKNOWN  \
count                 14460                   14460   
mean                      0                       0   
std                       0                       0   
min                       0                       0   
25%                       0                       0   
50%                       0                       0   
75%                       0                       0   
max                       0                       0   

                    ...                 COUNT CITIZEN STATUS TOTAL  \
count               ...                               14460.000000   
mean                ...                                  31.740249   
std                 ...                                  31.364111   
min                 ...                                   0.000000   
25%                 ...                                   5.000000   
50%                 ...                                  27.000000   
75%                 ...                                  52.000000   
max                 ...                                 106.000000   

       PERCENT CITIZEN STATUS TOTAL  COUNT RECEIVES PUBLIC ASSISTANCE  \
count                  14460.000000                      14460.000000   
mean                      76.230982                         10.521024   
std                       42.568309                          9.893284   
min                        0.000000                          0.000000   
25%                      100.000000                          1.000000   
50%                      100.000000                          8.000000   
75%                      100.000000                         19.000000   
max                      100.000000                         33.000000   

       PERCENT RECEIVES PUBLIC ASSISTANCE  COUNT NRECEIVES PUBLIC ASSISTANCE  \
count                        14460.000000                       14460.000000   
mean                             0.262372                          21.219225   
std                              0.177591                          21.833581   
min                              0.000000                           0.000000   
25%                              0.200000                           4.000000   
50%                              0.290000                          13.000000   
75%                              0.370000                          33.000000   
max                              1.000000                          73.000000   

       PERCENT NRECEIVES PUBLIC ASSISTANCE  COUNT PUBLIC ASSISTANCE UNKNOWN  \
count                         14460.000000                            14460   
mean                              0.499938                     

In [14]:
# convert budget to $s
indexed= budget_ses_attd_hs.set_index(['DATE', 'dbn'])

# start by dropping all duplicates
indexed = indexed.drop_duplicates()

# set zip as a categorical variable (and not as an integer variable)
# this is so the fixed effects regression works correctly
indexed['zip'] = indexed['zip'].astype('category')

# we drop these variables because they are redundant in our data frame
indexed = indexed.drop(['bloc', 'Location', 'Fiscal_Year'], axis=1)

# the values are strings starting with a $; convert them to float
def dollar_to_float(x):
    try:
        if x[0] == '$':
            return float(x[1:])
        else:
            return x
    except:
        return x

# We convert all the budget columns to floats
for c in indexed.columns:
    if re.search('S\d: Label ', c):
        indexed[c] = indexed[c].map(dollar_to_float)

indexed.head()

S1: Label a: FY14 Revised Base   \
DATE       dbn                                       
2015-09-09 14K071                          4721097   
2015-09-10 14K071                          4721097   
2015-09-11 14K071                          4721097   
2015-09-16 14K071                          4721097   
2015-09-17 14K071                          4721097   

                   S1: Label b: FSF & TL 09 C4E CTT   \
DATE       dbn                                         
2015-09-09 14K071                            4721097   
2015-09-10 14K071                            4721097   
2015-09-11 14K071                            4721097   
2015-09-16 14K071                            4721097   
2015-09-17 14K071                            4721097   

                   S1: Label c: FSF Over Formula  \
DATE       dbn                                     
2015-09-09 14K071                              0   
2015-09-10 14K071                              0   
2015-09-11 14K071                              0   
2015-09-16 14K071                              0   
2015-09-17 14K071                              0   

                   S2: Label d: FSF Preliminary  \
DATE       dbn                                    
2015-09-09 14K071                    4465933.24   
2015-09-10 14K071                    4465933.24   
2015-09-11 14K071                    4465933.24   
2015-09-16 14K071                    4465933.24   
2015-09-17 14K071                    4465933.24   

                   S2: Label a: FY14 Revised Base  \
DATE       dbn                                      
2015-09-09 14K071                         4721097   
2015-09-10 14K071                         4721097   
2015-09-11 14K071                         4721097   
2015-09-16 14K071                         4721097   
2015-09-17 14K071                         4721097   

                   S2: Label f: Register Change  \
DATE       dbn                                    
2015-09-09 14K071                    -255163.76   
2015-09-10 14K071                    -255163.76   
2015-09-11 14K071                    -255163.76   
2015-09-16 14K071                    -255163.76   
2015-09-17 14K071                    -255163.76   

                   S2: Label g: Teacher Salary Growth  \
DATE       dbn                                          
2015-09-09 14K071                                   0   
2015-09-10 14K071                                   0   
2015-09-11 14K071                                   0   
2015-09-16 14K071                                   0   
2015-09-17 14K071                                   0   

                   S2: Label h: New School Foundation  \
DATE       dbn                                          
2015-09-09 14K071                                   0   
2015-09-10 14K071                                   0   
2015-09-11 14K071                                   0   
2015-09-16 14K071                                   0   
2015-09-17 14K071                                   0   

                   S3: Label i: FY14 FSF at 100% S3: Foundation    \
DATE       dbn                                                      
2015-09-09 14K071                     5443475.28       $225000.00   
2015-09-10 14K071                     5443475.28       $225000.00   
2015-09-11 14K071                     5443475.28       $225000.00   
2015-09-16 14K071                     5443475.28       $225000.00   
2015-09-17 14K071                     5443475.28       $225000.00   

                                ...                COUNT CITIZEN STATUS TOTAL  \
DATE       dbn                  ...                                             
2015-09-09 14K071               ...                                        13   
2015-09-10 14K071               ...                                        13   
2015-09-11 14K071               ...                                        13   
2015-09-16 14K071               ...                                        13   
2015-09-17 14K071       

In [15]:
# just listing all the column names in the data frame
indexed.columns

Index([u'S1: Label a: FY14 Revised Base ',
       u'S1: Label b: FSF & TL 09 C4E CTT ', u'S1: Label c: FSF Over Formula',
       u'S2: Label d: FSF Preliminary', u'S2: Label a: FY14 Revised Base',
       u'S2: Label f: Register Change', u'S2: Label g: Teacher Salary Growth',
       u'S2: Label h: New School Foundation', u'S3: Label i: FY14 FSF at 100%',
       u'S3: Foundation  ', u'S3: Label d - Foundation',
       u'S3: Label i - Foundation', u'S3: Label j: FSF Final%',
       u'S4: Label d: FY14 FSF Initial', u'S4: AC Name: FSF (HS)',
       u'S4: AC Name: TL09 C4E CTT (HS)', u'S4: AC Name: Funds Over Formula',
       u'S5: TL SE Transitional Funding', u'As_Of_Date', u'zip',
       u'%_OF_ATTD_TAKEN', u'school_name', u'total_students', u'boro',
       u'COUNT PARTICIPANTS', u'COUNT FEMALE', u'PERCENT FEMALE',
       u'COUNT MALE', u'PERCENT MALE', u'COUNT GENDER UNKNOWN',
       u'PERCENT GENDER UNKNOWN', u'COUNT GENDER TOTAL',
       u'PERCENT GENDER TOTAL', u'COUNT PACIFIC ISLANDE

In [16]:
# save to a CSV for further analysis
indexed.to_csv('joined.csv')

In [22]:
from pandas.stats.plm import PanelOLS
# run a panel fixed-effects regression on variables

# We use a fixed-effects regression because we are trying to account
# for the fact that our variables do not encapsulate all of the
# observed variation in the output variable (% of attd taken). So,
# we want to separately control for each of the individual schools,
# and look at the results after accounting for them as separate
# entities.

# Further, we expect that there is a strong time-series correlation
# that we cannot easily difference out of our data, since we do not
# have troves of historical attendance data. Thus, we control for
# time effects as well.

# We can examine the regression F-statistic and related p-value to
# determine whether or not this regression is meaningful, and look
# at each variable's t-statistic and effect value to see how it
# interacts with the % of attd taken at 4 pm.
PanelOLS(y=indexed['%_OF_ATTD_TAKEN'], x=indexed[['PERCENT FEMALE', 'PERCENT RECEIVES PUBLIC ASSISTANCE', 'zip']], entity_effects=True, time_effects=True)


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <PERCENT FEMALE> + <PERCENT RECEIVES PUBLIC ASSISTANCE> + <zip>
             + <FE_01M448> + <FE_01M450> + <FE_01M509> + <FE_01M539> + <FE_01M696>
             + <FE_02M135> + <FE_02M139> + <FE_02M260> + <FE_02M280> + <FE_02M282>
             + <FE_02M288> + <FE_02M294> + <FE_02M296> + <FE_02M298>
             + <FE_02M300> + <FE_02M303> + <FE_02M305> + <FE_02M308> + <FE_02M316>
             + <FE_02M374> + <FE_02M376> + <FE_02M392> + <FE_02M393> + <FE_02M399>
             + <FE_02M400> + <FE_02M407> + <FE_02M408> + <FE_02M411> + <FE_02M412>
             + <FE_02M413> + <FE_02M414> + <FE_02M416> + <FE_02M418> + <FE_02M419>
             + <FE_02M420> + <FE_02M422> + <FE_02M425> + <FE_02M427> + <FE_02M432>
             + <FE_02M437> + <FE_02M438> + <FE_02M439> + <FE_02M449> + <FE_02M459>
             + <FE_02M489> + <FE_02M500> + <FE_02M507> + <FE_02M519>
             + <FE_02M520> + <FE_02M52